In [1]:
import os
import re
import zipfile
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from PIL import Image
import ast
import json
from glob import glob

In [2]:
use_size_down = True

In [3]:
class_names_kor = ['배경', '모름', '멧돼지사체','돼지사체', '돼지사체', '멧돼지사체', '큰부리까마귀', '흰배지빠귀', '유리딱새', '동고비', 
               '박새', '호랑지빠귀', '검독수리', '너구리', '설치류', '다람쥐', '사람']
class_names = ['bg','unknown', 'boar_dead', 'pig_dead','pig', 'boar', 'crow', 'bird_hinbejipagi', 'bird_ddaksae', 
               'bird_dongobi', 'bird_baksae', 'bird_horangjipagi', 'bird_eagle', 
               'racoon', 'rat', 'squrrel', 'human']

folder_pig = '/home/mvlab/Downloads/dataset/pig/'
folder_pig_annotation = folder_pig + 'annotation/'
os.path.isdir(folder_pig), os.path.isdir(folder_pig_annotation)

(True, True)

In [89]:
#['돼지사체', '사람', '조류', '멧돼지사체', '설치류', '포유류']
category_dict = {
    '큰부리까마귀': '조류',
    '까마귀': '조류',
 '너구리': '포유류',
 '다람쥐': '설치류',
 '동고비': '조류',
 '돼지사체': '포유류',
 '사람': '영장류',
 '멧돼지사체': '포유류', 
 '돼지': '포유류',
 '딱새': '조류',
 '유리딱새': '조류',
 '멧돼지': '포유류',
 '모름': '조류',
 '쥐': '설치류',
 '호랑지빠귀': '조류',
 '흰배지빠귀': '조류',
 '박새': '조류',
 '검독수리': '조류'
 }

In [61]:
dict_cls = dict()
for k, e in zip(class_names_kor, class_names):
    dict_cls[e] = k
    
dict_cls['undefine'] = '모름'
dict_cls['ndefine'] = '모름'
dict_cls['ndefin'] = '모름'
dict_cls['멧돼지'] = '멧돼지사체'
dict_cls['돼지'] = '돼지사체'
dict_cls['설치류'] = '쥐'
dict_cls

{'bg': '배경',
 'unknown': '모름',
 'boar_dead': '멧돼지사체',
 'pig_dead': '돼지사체',
 'pig': '돼지사체',
 'boar': '멧돼지사체',
 'crow': '큰부리까마귀',
 'bird_hinbejipagi': '흰배지빠귀',
 'bird_ddaksae': '유리딱새',
 'bird_dongobi': '동고비',
 'bird_baksae': '박새',
 'bird_horangjipagi': '호랑지빠귀',
 'bird_eagle': '검독수리',
 'racoon': '너구리',
 'rat': '설치류',
 'squrrel': '다람쥐',
 'human': '사람',
 'undefine': '모름',
 'ndefine': '모름',
 'ndefin': '모름',
 '멧돼지': '멧돼지사체',
 '돼지': '돼지사체',
 '설치류': '쥐'}

In [62]:
#list_annotation = glob(folder_pig_annotation + 'box1-10.*')
list_annotation = glob(folder_pig_annotation + '*.csv') + glob(folder_pig_annotation + '*.xlsx')
list_annotation = np.sort(list_annotation)
list_annotation

array(['/home/mvlab/Downloads/dataset/pig/annotation/box1-10.csv',
       '/home/mvlab/Downloads/dataset/pig/annotation/box1-11.csv',
       '/home/mvlab/Downloads/dataset/pig/annotation/box2-10.xlsx',
       '/home/mvlab/Downloads/dataset/pig/annotation/box2-11.csv',
       '/home/mvlab/Downloads/dataset/pig/annotation/box3-10.csv.xlsx',
       '/home/mvlab/Downloads/dataset/pig/annotation/box3-11.csv',
       '/home/mvlab/Downloads/dataset/pig/annotation/box4-10.csv',
       '/home/mvlab/Downloads/dataset/pig/annotation/box5-10.csv',
       '/home/mvlab/Downloads/dataset/pig/annotation/box5-11_csv.csv',
       '/home/mvlab/Downloads/dataset/pig/annotation/box6-10.csv',
       '/home/mvlab/Downloads/dataset/pig/annotation/box6-11.csv'],
      dtype='<U61')

In [63]:
def load_annotations(list_annotation):
    list_df = []
    for annotation in list_annotation:
        if annotation.endswith('xlsx'):
            df = pd.read_excel(annotation)
        else:
            df = pd.read_csv(annotation)
            
        print(annotation.split(os.sep)[-1], df.shape)
        list_df.append(df)
    return list_df        

In [64]:
list_df = load_annotations(list_annotation)

box1-10.csv (3451, 7)
box1-11.csv (7979, 7)
box2-10.xlsx (855, 7)
box2-11.csv (19648, 7)
box3-10.csv.xlsx (15014, 7)
box3-11.csv (21569, 7)
box4-10.csv (6519, 7)
box5-10.csv (1777, 7)
box5-11_csv.csv (197, 7)
box6-10.csv (1041, 7)
box6-11.csv (24408, 7)


In [65]:
#df = df6
df = pd.concat(list_df, axis=0, ignore_index=True)
df.shape

(102458, 7)

In [66]:
df = df[df['region_count']>0]
df.shape

(46221, 7)

In [67]:
path_images_0 = glob(folder_pig + '*/*/*.jpg')
path_images_1 = glob(folder_pig + '*/*/*/*.jpg')
path_images = path_images_0 + path_images_1

dict_path = dict() #5.0 GiB 7.9%
for path_image in path_images:
    image_file_name = path_image.split(os.sep)[-1]
    #print('image_file_name', image_file_name)
    dict_path[image_file_name] = path_image    

len(dict_path)
len(path_images), path_images[0]

(188539,
 '/home/mvlab/Downloads/dataset/pig/pig2/2020-10/geomex-pig02_2020-09-10_074418_temp-18.00_wat-97.10.jpg')

In [68]:
df['region_attributes'].unique()

array(['{"class":"돼지"}', '{"class":"사람"}', '{"class":"모름"}', '{}',
       '{"class":"멧돼지"}', '{"class":"흰배지빠귀"}', '{"class":"큰부리까마귀"}',
       '{"class":"다람쥐"}', '{"class":"호랑지빠귀"}', '{"class":"설치류"}',
       '{"class":"유리딱새"}', '{"class":"동고비"}', '{"class":"멧돼지사체"}',
       '{"class":"박새"}', '{"class":"검독수리"}', '{"class":"너구리"}',
       '{"class":undefined}', '{"class":"돼지사체"}', '{"class":"crow"}',
       '{"class":"pig"}'], dtype=object)

In [69]:
#전처리 해서 df 저장, 학습에 사용

list_cls = []
list_valid = []
pre = "{'class':'"
end = ["'}'"]
for v in df['region_attributes'].values:
    cls = '{}'
    is_valid = len(v) > 2
    if is_valid:        
        cls = v[len(pre):-2]
        #cls.endswith(end[0])
        if cls in dict_cls.keys():
            cls = dict_cls[cls]
            #print(v, cls)
    
    list_cls.append(cls)
    list_valid.append(is_valid)
    #break

In [70]:
np.array(list_valid).mean()

0.9995456610631531

In [71]:
len(df), len(list_cls), np.unique(list_cls)

(46221,
 46221,
 array(['{}', '검독수리', '너구리', '다람쥐', '동고비', '돼지사체', '멧돼지사체', '모름', '박새',
        '사람', '유리딱새', '쥐', '큰부리까마귀', '호랑지빠귀', '흰배지빠귀'], dtype='<U6'))

In [72]:
df['region_attributes'] = list_cls
df = df[list_valid]
df.shape

(46200, 7)

In [76]:
df['region_attributes'].unique()

array(['돼지사체', '사람', '모름', '멧돼지사체', '흰배지빠귀', '큰부리까마귀', '다람쥐', '호랑지빠귀',
       '쥐', '유리딱새', '동고비', '박새', '검독수리', '너구리'], dtype=object)

In [90]:
df['category'] = df['region_attributes'].replace(category_dict)

In [91]:
df['category'].unique()

array(['포유류', '영장류', '조류', '설치류'], dtype=object)

In [92]:
df.head()

,filename,file_size,file_attributes,region_count,region_id,region_shape_attributes,region_attributes,category
0,geomex-pig01_2020-09-03_125416_temp-19.70_wat-...,362254,{},2,0,"{""name"":""rect"",""x"":583,""y"":463,""width"":453,""he...",돼지사체,포유류
1,geomex-pig01_2020-09-03_125416_temp-19.70_wat-...,362254,{},2,1,"{""name"":""rect"",""x"":560,""y"":563,""width"":224,""he...",사람,영장류
2,geomex-pig01_2020-09-03_125706_temp-19.80_wat-...,365469,{},2,0,"{""name"":""rect"",""x"":907,""y"":332,""width"":211,""he...",사람,영장류
3,geomex-pig01_2020-09-03_125706_temp-19.80_wat-...,365469,{},2,1,"{""name"":""rect"",""x"":584,""y"":457,""width"":369,""he...",사람,영장류
4,geomex-pig01_2020-09-03_125956_temp-20.70_wat-...,362131,{},2,0,"{""name"":""rect"",""x"":339,""y"":128,""width"":107,""he...",사람,영장류


In [93]:
df.to_csv('all_2021-1-2.csv')

In [43]:
annotation = dict()
object_width_list = []
for i in range(len(df)):        
    sample_row = df.iloc[i]
    filename = sample_row['filename']    
    region_id = sample_row['region_id']
    box = sample_row['region_shape_attributes']    
    cls = sample_row['region_attributes'] 
    
    if '(' in filename:
        print('()', filename)
    try:
        box_map = ast.literal_eval(box)    
        
    except:
        print('except filename',i, filename)
        print('box', box)        
        continue
    
    if cls in class_names_kor:
        cls_num = class_names_kor.index(cls)
    else:
        print('no key', cls)
        break
        
    x0 = box_map['x']
    y0 = box_map['y']
    width = box_map['width']
    height = box_map['height']    
   
    object_width_list.append(width)    
    bbox = [cls_num, x0, y0, x0 + width, y0 + height]
    
    if filename not in annotation.keys():
        annotation[filename] = []
   
    annotation[filename].append(bbox)     

() geomex-pig04_2020-09-12_085151_temp-17.50_wat-99.90 (1).jpg
() geomex-pig04_2020-09-12_085151_temp-17.50_wat-99.90 (1).jpg
() geomex-pig04_2020-09-12_085151_temp-17.50_wat-99.90 (1).jpg
() geomex-pig04_2020-10-30_092128_temp-11.50_wat-51.40 (1).jpg
() geomex-pig04_2020-10-30_092128_temp-11.50_wat-51.40 (1).jpg


In [44]:
len(annotation)

13165

In [21]:
def load_pig(stride=1):
    input_list = []
    bbox_list = []
    path_list = []
    for path_image in annotation:
        if stride > 1 and np.random.randint(0, stride) > 1:
            continue
            
        cls_bbox = annotation[path_image]
        cls_bbox = np.array(cls_bbox).reshape([-1, 5])
        cls = cls_bbox[:, 0:1]
        bbox = np.array(cls_bbox[:, 1:])
              
        full_path = dict_path[path_image]
        if os.path.isfile(full_path):
            img = Image.open(full_path)                           
            scale = np.array((img.width, img.height, img.width, img.height)).reshape((1, 4))            
            #print(key, cls, cls_bbox.dtype, cls_bbox, 'wh',img.width, img.height)
            
            if use_size_down:
                w, h = img.size
                img = img.resize((w//2, h//2))
            img_arr = np.array(img)        
            
            bbox_norm = bbox.astype(np.float) / scale.astype(np.float)
            cls = np.array(cls).reshape((-1, 1))
            cls_bbox_norm = np.concatenate((cls, bbox_norm), axis=1)
                
            input_list.append(img_arr)
            bbox_list.append(cls_bbox_norm)
            path_list.append(full_path)
            if len(input_list)%100==0:        
                print(len(annotation), len(input_list), len(bbox_list))   
            if len(input_list) > 15000:
                print('stop', len(input_list))
                break
        else:
            print('not exist', path_image)
            print('not exist full_path', full_path)

    print(len(input_list), len(bbox_list))
    return input_list, bbox_list
    
input_list_pig, bbox_list_pig = load_pig(stride=100)

13165 100 100
13165 200 200
270 270


In [22]:
boxes = np.concatenate(bbox_list_pig, 0)
boxes.shape, np.sum(boxes[:, 0]>3)

((953, 5), 691)

In [23]:
len(input_list_pig)

270

In [24]:
def crop_bbox_image(image, boxes):
    img_objects = []
    image = np.array(image)
    for box in boxes:        
        box = box.astype(np.int)
        x1, y1, x2, y2 = box
        w = x2 - x1
        h = y2 - y1        
        crop_image_arr = image[y1:y2, x1:x2]
        ch, cw, cc = crop_image_arr.shape
        if ch>1 and cw>1:
            img_objects.append(crop_image_arr)
        else:
            print('crop_bbox_image', x2-x1, y2-y1, 'crop_image_arr.shape', crop_image_arr.shape)
        
    return img_objects
    
    
def attach_crop_image(image, boxes, max_crop=200):
        
    crop_bbox_arr = crop_bbox_image(image, np.array(boxes)[:max_crop])
    bbox_k = len(crop_bbox_arr)
    max_col = 30
    
    if bbox_k > 0:
        img_h, img_w, img_c = image.shape
        object_img_w = img_w//bbox_k        
        resize_h = img_h // 8
        resize_w = img_w // bbox_k  
        resize_w = min(max(resize_w, img_w//max_col), img_w//8)
        
        footer_h = resize_h * (1 + (bbox_k-1)//max_col)
        footer = np.zeros((footer_h, img_w, img_c), np.uint8)
        
        for i in range(min(bbox_k, max_crop)):
            crop_arr = crop_bbox_arr[i]
            crop_img = Image.fromarray(crop_arr)                
            crop_img = crop_img.resize((resize_w, resize_h))
            crop_arr_resized = np.array(crop_img)
            offset_y = (i//max_col) * resize_h
            offset_x = (i%max_col) * resize_w
            footer[offset_y:offset_y+resize_h, offset_x:offset_x+resize_w] = crop_arr_resized

        seperate_line = np.zeros_like(footer[:2])
        image = np.concatenate((image, seperate_line, footer), axis=0)    
    return image    

In [25]:
def visualize_detections_simple(
    image, boxes, classes, figsize=(12, 12), linewidth=1, color=[0, 0, 1]
):
    """Visualize Detections"""
    image = np.array(image, dtype=np.uint8)    
    
    img_h, img_w, img_c = image.shape
    
    image = attach_crop_image(image, boxes, max_crop=100)
    
    plt.figure(figsize=figsize)
    plt.axis("off")
    plt.imshow(image)
    ax = plt.gca()
    boxes_width = boxes[:, 2] - boxes[:, 0]
    boxes_height = boxes[:, 3] - boxes[:, 1]
    box_min_width = np.min(boxes_width)
    box_max_width = np.max(boxes_width)
    title = str.format('(%dx%d) %d box, width:%d ~ %d' 
                       %(img_h, img_w, len(boxes), box_min_width, box_max_width))
    plt.title(title)
    for box, cls in zip(boxes, classes):
        x1, y1, x2, y2 = box        
        w, h = x2 - x1, y2 - y1
        #color = edgecolors[int(cls)]
        color = (1, 1, 1)
        patch = plt.Rectangle(
            [x1, y1], w, h, fill=False, edgecolor=color, linewidth=linewidth
        )
        ax.add_patch(patch)
        if len(boxes) < 70:
            score_txt = class_names[int(cls)]
            ax.text(x1, y1, score_txt, bbox={"facecolor": [1,1,0], "alpha": 0.4}, clip_box=ax.clipbox, clip_on=True,)
        
    plt.show()
    return ax

def display_data(X, BBOX, stride=1):
    for i in range(len(X)):
        if i%stride==0:
            img_arr = X[i]
            sample_box = BBOX[i]
            label = sample_box[:, 0]
            bbox = sample_box[:, 1:]

            h, w, c = img_arr.shape
            scale = np.array((w, h, w, h))
            scale = np.reshape(scale, (1, 4))
            bbox_norm = bbox.astype(np.float) * scale.astype(np.float)
            #print('bbox_norm', bbox, bbox_norm)
            print(i, np.unique(label))
            ax = visualize_detections_simple(img_arr,bbox_norm,label)

s = 0#len(input_list_pig)//2
e = -1
display_data(input_list_pig[s:e], bbox_list_pig[s:e], stride=10)